# The ASDF Palindrome Challenge

My colleague Jimmie nerd-sniped me with the question _"what's the longest Finnish palindrome that can be made on one row of a keyboard?"_

Let's explore this question a bit.

On the top and bottom rows of a QWERTY keyboard, the letters `Q`, `W`, `Z`, `X`, `C` and `B` are really challenging. The middle row contains fewest letters foreign to Finnish. The letters are `A`, `S`, `D`, `F`, `G`, `H`, `J`, `K`, `L`, `Ö` and `Ä` in the Finnish keyboard layout.

Long palindromes are easier to find than short ones. It's easiest to get started in the middle, but it becomes difficult to find a terminating pair of words for the ends.

So let's start with a long palindrome which uses a subset of the allowed letters. That's a trivial task: just find a short word which can form a valid sentence by itself: _"Aja!"_ ("Drive!"). Then just repeat that:

>**Aja! Aja! Aja! Aja! Aja! Aja! Aja! Aja! Aja! Aja! Aja! Aja! Aja! Aja! Aja! Aja! Aja! Aja! Aja! Aja!**

We can also spice it with _"ajaja"_ ("driver") and _"ala!"_ (begin!):

>**Ala! Ala, ajaja, aja! Aja! Aja, ajaja, ala! Aja! Ala, ajaja, aja! Aja! Aja, ajaja, ala! Ala!**

You get the idea. Similarly, the shortest palindrome with a subset of the chosen letters is probably:

>**Aja!**

---

## Help from computers

At this point, let's create a palindrome verifier and use it for palindromes we come up with:

In [1]:
from IPython.core.magic import register_line_cell_magic
from IPython.core.display import HTML
import re

ALPHABET = 'asdfghjklöä'

@register_line_cell_magic
def check(line, cell=None):
    if cell:
        # e.g. %% check
        #      Onko tämä palindromi?
        #      # Is this a palindrome?
        palindrome = '\n'.join(l for l in cell.split('\n')
                               if not l.startswith('#')).strip()
    else:
        # e.g. "%check  Onko tämä palindromi?  # Is this a palindrome?"
        palindrome = line.split('#')[0]
    # get only letters, in lower case, e.g. "onkotämäpalindromi"
    letters = [c for c in palindrome.lower() if c.isalpha()]
    if letters == list(reversed(letters)):
        # it is a palindrome
        output_lines = [f'<blockquote style="white-space: pre"><strong>{palindrome}</strong></blockquote>']
    else:
        # it's not a palindrome
        output_lines = [f'<p style="color: var(--jp-error-color0)">Not a palindrome: {palindrome}</p>']
    # find ASDF letters found in the palindrome, e.g. "ADKLÄ"
    contains = ''.join(c for c in ALPHABET if c in letters).upper()
    output_lines.extend(['<ul>', f'<li>Contains: {contains}</li>', '</ul>'])
    # find ASDF letters missing from the palindrome, e.g. "SFGHJÖ"
    lacks = ''.join(c for c in ALPHABET if c not in letters).upper()
    if lacks:
        output_lines.insert(-1, f'<li>Lacks: {lacks}</li>')
    # find non-ASD letters in the palindrome, e.g. "IMNOPRT"
    extra = ''.join(sorted(set(letters).difference(ALPHABET))).upper()
    if extra:
        output_lines.insert(-1, f'<li>Has extra letters: {extra}</li>')
    return HTML(''.join(output_lines))    

Let's do a few tests to see that this works as expected:

In [2]:
%check Innostunut sonni

In [3]:
%check  A non-palindrome on a single line

In [4]:
%%check

A non-palindrome
on multiple lines

In [5]:
%check  Ei-palindromi ja käännös  # A non-palindrome with a translation

In [6]:
%%check

Ei-palindromi usealla rivillä
ja sen käännös.

# A non-palindrome on multiple lines
# and its translation.

In [7]:
%%check

"Allasi on ase",
sanoi Salla.

# "There's a weapon underneath you",
# said Sally.

Ok, our verifier seems to work. Let's get to work!

---

## Allow no repeats

Since _"Aja! Aja! Aja! Aja! Aja!"_ is a little stupid for a palindrome,
let's say that no word (or sequence of words) can appear twice in a row. We can still quite easily play with the "easy" letters:

In [8]:
%check  Äläkä, jäkälä!  # And don't, you lichen!

A good technique at this point is to expand the palindrome at the middle:

In [9]:
%check  Äläkä jaa, jäkälä!  # And don't share, you lichen!

In [10]:
%check  Äläkä jaa saajaa! Saa jäkälä!  # And don't share the receiver! Receive a lichen!

In [11]:
%check  Äläkä jaa saajalla. Jaa, saa jäkälä!  # And don't divide by the receiver! Share, receive a lichen!

In [12]:
%check  Äläkä jaa saajalla. Kakalla jaa, saa jäkälä!  # And don't divide by the receiver! Divide by poop, receive a lichen!

With more work, we could probably find different palindromes which don't fail to make sense this miserably.

---

## Use every key on the ASDF row

To make the challenge really interesting, I would adjust Jimmie's idea a bit: use each of the letters on the middle row of the keyboard at least once.

My usual methods for creating palindromes involve using either
- just the brain,
- the brain, pen, and paper,
- the brain, keyboard, and screen, or
- the brain, keyborad, screen, and a queryable electronic Finnish dictionary.

For this challenge, let's extend that toolset with the Python language and Internet resources.

The [Voikko](https://voikko.puimula.org/) library provides spell checking and hyphenation for LibreOffice and other Open Source word processors.
Surprisingly, the [Voikko Python package](https://pypi.org/project/voikko/) also includes a word inflection tool and a 46000-word dictionary.
So let's install it and create a big list of inflected words:

In [13]:
!pip install -q voikko

In [14]:
from pathlib import Path
import pickle
from voikko.inflect_word import generate_forms, WORD_CLASSES
len(WORD_CLASSES)

46548

In [15]:
WORD_FORM_CACHE = Path('word-forms.pickle')
if WORD_FORM_CACHE.is_file():
    with WORD_FORM_CACHE.open('rb') as cache:
        word_form_lists = pickle.load(cache)
else:
    word_form_lists = [generate_forms(word_class) for word_class in WORD_CLASSES]
    with WORD_FORM_CACHE.open('wb') as cache:
        pickle.dump(word_form_lists, cache)

all_forms = [form_variant.lower()
             for word_interpretation_form_list in word_form_lists
             for word_forms in word_interpretation_form_list
             for form_variants in word_forms.values()
             for form_variant in form_variants]

len(all_forms)

1085562

So we conveniently got a list of million Finnish words!
Let's then filter that to only words which contain letters from the ASDF row of the keyboard:

In [16]:
def filter_asdf(words):
    return {word for word in words
            if not set(word.lower()).difference(ALPHABET)}

asdf_words = filter_asdf(all_forms)
len(asdf_words)

441

Only 441 words! That gives us very little material for creating palindromes!
Let's take a look at the actual words:

In [17]:
from textwrap import wrap

def list_words(words):
    for line in wrap(' '.join(sorted(words)), 120):
        print(line)
        
list_words(asdf_words)

aada aadaa aadalla aadassa ada adaa adalla adassa ahdas ajaa ajaja ajajaa ajajalla ajajassa ajalla ajassa akaa akaalla
akaassa akalla akassa akka akkaa akkala akkalaa akkalalla akkalassa ala alaa alalla alaska alaskaa alaskalla alaskassa
alassa alf alfa alfaa alfalfa alfalfaa alfalfalla alfalfassa alfalla alfassa alkaa alkaja alkajaa alkajalla alkajassa
allah allakalla allakassa allakka allakkaa allas aska askaa askalla askassa aslak dada dadaa dadalla dadassa dahl dallas
dödö dödöjä dödöllä dödössä dödöä dösä dösällä dösässä dösää falk flada fladaa fladalla fladassa flagg flash föglö
föglöllä föglössä gaala gaalaa gaalalla gaalassa göös haag haaga haagaa haagalla haagassa haahka haahkaa haahkalla
haahkassa haahla haahlaa haahlalla haahlassa haalla haaska haaskaa haaskalla haaskassa haassa hahla hahlaa hahlalla
hahlassa haka hakaa hakala hakalaa hakalalla hakalassa hakkaaja hakkaajaa hakkaajalla hakkaajassa hal haljakalla
haljakassa haljakka haljakkaa haljas hall halla hallaa hallalla 

To make our work easier, we can still trim down the list.
There are letter combinations which obviously don't work when reversed,
especially within this limited set of words.

Take a look e.g. at the word "SÄHKÖ" and its inflections.
The reversed "ÖKHÄS" looks challenging. We must consider the five possible splitting points:
- "ÖKHÄS": there are no words beginning with "ÖKH" in Finnish
- "Ö | KHÄS": also, no words beginning with "KH"
- "ÖK | HÄS": no words ending with "ÖK"
- "ÖKH | ÄS": no words ending with "KH"
- "ÖKHÄ | S": no words ending with "KHÄ" (we could use "HÄ?", but then we're back at the ending "ÖK")

So we can safely say that "SÄHKÖ" and its inflections are not usable for a palindrome.
With similar reasoning, we can reject a bunch of other words as well:

In [18]:
def remove_unusable_words(words):
    return {
        word for word in words
        if not word.startswith(
            ('ahdas', 'alf', 'alka',
             'dahl',
             'falk', 'flada', 'flagg', 'flash', 'föglö',
             'gaalalla', 'gaalassa', 'göös',
             'haahka', 'haahla', 'haaskaa', 'haaskalla', 'hahla', 'haljas', 'hädä', 'häkköjä', 'hässäkk', 'häädö', 'hölkkä',
             'jaffaa', 'jaffalla', 'jaffassa',
             'kafka', 'kahlaaja', 'kaskaa', 'klaas', 'lafka', 'lahjaa', 'lahjakk', 'lahjalla', 'lahjassa', 'lähdö',
             'safka', 'skaala', 'sählääjä', 'säädö', 'sähkö'))}

asdf_good_words = remove_unusable_words(asdf_words)

There are also some obvious words missing from this list (worth a bug report to the Voikko Python library?):

In [19]:
asdf_good_words.update({'lagaa', 'lagaaja', 'kaj', 'jaa', 'saa', 'aja', 'saska'})

list_words(sorted(asdf_good_words))

aada aadaa aadalla aadassa ada adaa adalla adassa aja ajaa ajaja ajajaa ajajalla ajajassa ajalla ajassa akaa akaalla
akaassa akalla akassa akka akkaa akkala akkalaa akkalalla akkalassa ala alaa alalla alaska alaskaa alaskalla alaskassa
alassa allah allakalla allakassa allakka allakkaa allas aska askaa askalla askassa aslak dada dadaa dadalla dadassa
dallas dödö dödöjä dödöllä dödössä dödöä dösä dösällä dösässä dösää gaala gaalaa haag haaga haagaa haagalla haagassa
haalla haaska haaskassa haassa haka hakaa hakala hakalaa hakalalla hakalassa hakkaaja hakkaajaa hakkaajalla hakkaajassa
hal haljakalla haljakassa haljakka haljakkaa hall halla hallaa hallalla hallassa häkkä häkkää häkä häkällä häkässä häkää
häläjää hässäkällä hässäkässä hää häällä häässä höhlä höhlällä höhlässä höhlää hölkällä hölkässä höllä höllällä höllässä
höllää hölö hölöjä hölöllä hölössä hölöä höskä höskällä höskässä höskää jaa jaala jaalaa jaalalla jaalassa jaffa jaja
jajaa jajalla jajassa jakaa jakaja jakajaa jakajall

To double check for other omissions, let's compare to a dictionary of Finnish words from [Kotus](http://kaino.kotus.fi/sanat/nykysuomi/):

In [20]:
KOTUS_SANALISTA_URL = 'http://kaino.kotus.fi/sanat/nykysuomi/kotus-sanalista-v1.zip'
KOTUS_SANALISTA_XML = 'kotus-sanalista_v1/kotus-sanalista_v1.xml'

# Cache the download so we only do it once even if we run this notebook multiple times
!pip install -q --user requests-cache
import requests_cache
requests_cache.install_cache('cache')

import requests
import io
from zipfile import ZipFile

zip_content = requests.get(KOTUS_SANALISTA_URL).content
with ZipFile(io.BytesIO(zip_content)) as kotus_zip:
    with kotus_zip.open(KOTUS_SANALISTA_XML) as kotus_xml:
        kotus_asdf_words = remove_unusable_words(
            filter_asdf({l[7:].decode('utf-8').split('<', 1)[0]
                        for l in kotus_xml if l.startswith(b'<st><s>')}))
len(kotus_asdf_words)

107

In [21]:
list_words(kotus_asdf_words.difference(asdf_good_words))

ah ahaa alas alhaalla alla ha haa hah hajalla hä häh häälahja hössäkkä ja jaaha jaha jahka jöö kajal kaks kalkkaa kallas
kas käkö kökkö köö laakajäkälä sa saakka sadas sä ähkää älkää älä


Let's add some of those to the Voikko set of words:

In [22]:
asdf_good_words.update({'ah', 'ahaa', 'alas', 'alla',
                        'dj',
                        'faksaa', 'faksaaja',
                        'ha', 'haa', 'hah', 'hajalla', 'hä', 'häh', 'hössäkkä',
                        'ja', 'jaaha', 'jaha', 'jöö', 'jööllä', 'jöössä',
                        'kajal', 'kaks', 'kallas', 'kas', 'käkö', 'kökkö',
                        'laakajäkälä', 'saakka', 'sadas', 'sä',
                        'ähkää', 'älkää', 'älä'})

len(asdf_good_words)

378

In [23]:
list_words(asdf_good_words)

aada aadaa aadalla aadassa ada adaa adalla adassa ah ahaa aja ajaa ajaja ajajaa ajajalla ajajassa ajalla ajassa akaa
akaalla akaassa akalla akassa akka akkaa akkala akkalaa akkalalla akkalassa ala alaa alalla alas alaska alaskaa
alaskalla alaskassa alassa alla allah allakalla allakassa allakka allakkaa allas aska askaa askalla askassa aslak dada
dadaa dadalla dadassa dallas dj dödö dödöjä dödöllä dödössä dödöä dösä dösällä dösässä dösää faksaa faksaaja gaala
gaalaa ha haa haag haaga haagaa haagalla haagassa haalla haaska haaskassa haassa hah hajalla haka hakaa hakala hakalaa
hakalalla hakalassa hakkaaja hakkaajaa hakkaajalla hakkaajassa hal haljakalla haljakassa haljakka haljakkaa hall halla
hallaa hallalla hallassa hä häh häkkä häkkää häkä häkällä häkässä häkää häläjää hässäkällä hässäkässä hää häällä häässä
höhlä höhlällä höhlässä höhlää hölkällä hölkässä höllä höllällä höllässä höllää hölö hölöjä hölöllä hölössä hölöä höskä
höskällä höskässä höskää hössäkkä ja jaa jaaha jaala jaalaa

---

## Word matcher

We need one more tool: a quick way to search for matching words given some criteria.

In [24]:
from IPython.core.magic import register_line_magic

@register_line_magic
def where(callback):
    list_words(w for w in asdf_good_words if eval(callback))

We conveniently found a beginning and end for the palindrome, so let's start expanding in the middle.
Let's try with **Äläkä jaksa ...aska Jäkälä** and see what we can find for the middle:

In [25]:
%where w.endswith('aska')

alaska aska haaska jaska saska


In [26]:
%check  Äläkä jaksa, Jaska Jäkälä!  # And spare me, Jack the Lichen!

In [27]:
%where w.startswith('la')  # äläkä jaksa, ... [AL]aska-jäkälä

laaja laajaa laajalla laajassa laaka laakaa laakajäkälä lada ladaa ladalla ladassa lagaa lagaaja lahja lahjakas lakalla
lakassa lakka lakkaa


In [28]:
%check  Äläkä jaksa, laaja Alaska-jäkälä!  # And spare me, broad Alaskan lichen!

In [29]:
%check  Äläkä jaksa laajalla, ja Alaska-jäkälä!  # And spare me on a broad, and Alaskan lichen!

---

## The first complete one

Using these tools, I explored for a while and came up with this one:

In [30]:
%%check

Hallas ällö dödöllä.
Äläkä, Jaffa, jaa salaa!
Gaala saa, Jaffa.
Jäkälä ällö, dödölläs Allah.

# Your disgusting frost with deodorant.
# And don't, Jaffa, share secretly!
# Get a gala, Jaffa.
# A disgusting lichen, Allah with your deodorant.

Let's see if we could get rid of the repetitions by putting F/G words in the middle.

In [31]:
%where 'f' in w or 'g' in w

faksaa faksaaja gaala gaalaa haag haaga haagaa haagalla haagassa jaffa lagaa lagaaja saaga saagaa saagalla saagassa saga
sagaa sagalla sagassa


In [32]:
%check  ...Haaska, faksaa h...  # ...Carcass, fax a h...

In [33]:
%where w.startswith('h') and 'g' in w

haag haaga haagaa haagalla haagassa


In [34]:
%check  ...Saagaa, haaska, faksaa! Haagaa s...  # Carcass, fax a saga! District of Haaga s...

---

## Another helper: how many ASDF row letters does a palindrome use?

This helper function will make it easier to filter down candidate words to those which have enough of the remaining required ASDF letters:

In [35]:
def count_letters(letters, text):
    return len(set(letters).intersection(text))

And here's how to use it:

In [36]:
%where w.startswith('s') and count_letters('djlöä', w) >= 2

saajalla sahaajalla sähäjää sähäkköjä sähäkällä säkällä sälä sälällä sälässä sälää sälö sälöjä sälöllä sälössä sälöä
säällä sököjä sököllä sökössä sököä sössöjä sössöllä sössössä sössöä


In [37]:
%check  ...ä. Älä saagaa, haaska, faksaa! Haagaa, sälää...  # ...ä. Carcass, don't fax a saga! District of Haaga, odds and ends...

In [38]:
%where w.endswith('ä') and count_letters('djö', w) >= 1 and w[-3:-1] not in ['ll', 'ss']

dödöjä dödöä dösä dösää häläjää höhlä höhlää höllää hölöjä hölöä höskä höskää hössäkkä jäkälä jäkälää jää jäädä kähäjää
köhä köhäjää köhää laakajäkälä läjä läjää löllöjä löllöä sähäjää sähäkköjä sälöjä sälöä sököjä sököä sössöjä sössöä
ähäjää ällöjä ällöä äläjää äläkköjä öllölä öllölää


In [39]:
%check ...jäkälä. Älä saagaa, haaska, faksaa! Haagaa. sälä. Äläkä j...

# ...lichen. Carcass, don't fax a saga! District of Haaga, odds and ends. And don't j...

In [40]:
%where w.startswith('j')

ja jaa jaaha jaala jaalaa jaalalla jaalassa jaffa jaha jaja jajaa jajalla jajassa jakaa jakaja jakajaa jakajalla
jakajassa jaksaa jalalla jalas jalassa jalka jalkaa jaska jaskaa jaskalla jaskassa jäkälä jäkälällä jäkälässä jäkälää
jää jäädä jäällä jäässä jöö jööllä jöössä


In [41]:
%check ...höllää! Älä saagaa, haaska, faksaa! Haagaa, sälää, ällö h...

# ...loosen! Carcass, don't fax a saga! District of Haaga, odds and ends, a disgusting h...

In [42]:
%where w.startswith('h') and count_letters('dj', w) >= 1

hajalla hakkaaja hakkaajaa hakkaajalla hakkaajassa haljakalla haljakassa haljakka haljakkaa häläjää hölöjä


---

## Almost there

We're still lacking a `D`, but this is coming close already:

In [43]:
%%check

Aja, akka, höllää!
Älä saagaa, haaska, faksaa!
Haagaa, sälää, ällö hakkaaja.

# Drive, woman, loosen!
# Carcass, don't fax a saga!
# District of Haaga, odds and ends, the disgusting beater.

In [44]:
%where 'd' in w

aada aadaa aadalla aadassa ada adaa adalla adassa dada dadaa dadalla dadassa dallas dj dödö dödöjä dödöllä dödössä dödöä
dösä dösällä dösässä dösää jäädä lada ladaa ladalla ladassa saada sadas


In [45]:
%check ...Ladaa aja, akka, höllää! Älä saagaa, haaska, faksaa! Haagaa, sälää, ällö hakkaaja-Aada l...

# ...Drive a Lada, woman, loosen! Carcass, don't fax a saga! District of Haaga, odds and ends, the disgusting beater Ada l...

In [46]:
%where w.startswith('l')

laaja laajaa laajalla laajassa laaka laakaa laakajäkälä lada ladaa ladalla ladassa lagaa lagaaja lahja lahjakas lakalla
lakassa lakka lakkaa läjä läjällä läjässä läjää löllö löllöjä löllöllä löllössä löllöä


---

## Two additional solutions

And through this process, we arrive at two candidates for an ASDFGHJKLÖÄ palindrome with actual Finnish words and the faint smell of a possibility of some logic:

In [47]:
%%check

Ajaa Ladaa.
Aja, akka, höllää!
Älä saagaa, haaska, faksaa!
Haagaa, sälää, ällö hakkaaja-Aada laaja.

# Drives a Lada.
# Woman, drive, loosen!
# Carcass, don't fax a saga!
# District of Haaga, odds and ends, the disgusting and broad beater Ada.

In [48]:
%%check

Aada, Ladaa aja, akka, höllää!
Älä saagaa, haaska, faksaa!
"Haagaa, sälää!", ällö hakkaaja-Aada Ladaa.

# Ada, drive a Lada, woman, loosen!
# Carcass, don't fax a saga!
# "District of Haaga, odds and ends!", the disgusting beater Ada to the Lada.